In [1]:
import os

%pwd

'c:\\Code\\Machine-Learning\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Code\\Machine-Learning\\Kidney-Disease-Classification-Deep-Learning-Project'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = (
    "https://dagshub.com/DhruvJais01/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
)
os.environ["MLFLOW_TRACKING_USERNAME"] = "DhruvJais01"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "02273b0560c7a8898cfff4ae9e458e1e1eae2899"

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: list

In [7]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_evaluate_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/DhruvJais01/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [9]:
import mlflow
import mlflow.keras
import tensorflow as tf
from mlflow.models.signature import infer_signature

In [13]:
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.3)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs,
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model, "model", registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")


In [11]:
try:
    config = ConfigurationManager()
    evaluate_config = config.get_evaluate_config()
    evaluation = Evaluation(evaluate_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-27 15:36:04,163 : INFO : common : YAML file loaded successfully: config\config.yaml]
[2025-10-27 15:36:04,167 : INFO : common : YAML file loaded successfully: params.yaml]
[2025-10-27 15:36:04,168 : INFO : common : Directory created successfully: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 35.9189 - accuracy: 0.4820
[2025-10-27 15:36:18,183 : INFO : common : JSON file saved successfully: scores.json]
1/1 [==============================] - 2s 2s/step
[2025-10-27 15:36:24,086 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\91630\AppData\Local\Temp\tmpixff_nne\model\data\model\assets
[2025-10-27 15:36:24,633 : INFO : builder_imp

c:\Code\Machine-Learning\Kidney-Disease-Classification-Deep-Learning-Project\.venv\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/10/27 15:38:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
